In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

![](../figs/dp-policy-iteration.png)

1. Policy Evaluation: Determining the value function for the current policy.
2. Policy Improvement: Adjusting the policy to select actions that maximize the expected value, based on the evaluated value function.

In [3]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.0001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

In [16]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI environment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    # Policy iteration counter
    k = 0

    # Policy iteration
    while True:
        k += 1
        
        # Policy evaluation
        V = policy_eval_fn(policy, env, discount_factor)

        # Will be set to False if we make any changes to the policy
        convergence = True

        # For each state s, find the action a that maximizes the expected value.
        for s in range(env.nS):
            # Greedy search the actions Q(s,a)
            Q_sa = np.zeros(env.nA)
            for a in range(env.nA):
                for transition_prob, next_state, reward, done in env.P[s][a]:
                    Q_sa[a] += transition_prob * (reward + discount_factor * V[next_state])
            
            # Find the best actions for the state
            best_actions = np.where(Q_sa == np.max(Q_sa))[0]
            
            # Compare with the current policy
            if not np.array_equal(best_actions, np.where(policy[s] == np.max(policy[s]))[0]):
                # Update the policy to be greedy with respect to Q(s, a)
                policy[s] = 0  # Reset the policy for state s
                policy[s, best_actions] = 1 / len(best_actions)  # Uniform distribution over best actions
                convergence = False  # Mark that the policy has changed

        if convergence:
            print("Number of Policy Improvement Iterations: {}".format(k))
            print("")
            return policy, V


In [17]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

def get_max_actions(policy_row):
    """Returns a list of actions with the maximum probability."""
    max_prob = np.max(policy_row)
    return [i for i, prob in enumerate(policy_row) if prob == max_prob]

# Assuming 'policy' is your policy array and 'env' is your GridworldEnv
grid_policy = np.array([get_max_actions(row) for row in policy], dtype=object)

# Use repr to get the string representation of the array
grid_policy_str = repr(np.reshape(grid_policy, env.shape))

# Remove "list" from the string
grid_policy_str = grid_policy_str.replace("list", "")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(grid_policy_str)

Number of Policy Improvement Iterations: 3

Policy Probability Distribution:
[[0.25 0.25 0.25 0.25]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   1.  ]
 [0.   0.   0.5  0.5 ]
 [1.   0.   0.   0.  ]
 [0.5  0.   0.   0.5 ]
 [0.25 0.25 0.25 0.25]
 [0.   0.   1.   0.  ]
 [1.   0.   0.   0.  ]
 [0.25 0.25 0.25 0.25]
 [0.   0.5  0.5  0.  ]
 [0.   0.   1.   0.  ]
 [0.5  0.5  0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.25 0.25 0.25 0.25]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
array([[([0, 1, 2, 3]), ([3]), ([3]), ([2, 3])],
       [([0]), ([0, 3]), ([0, 1, 2, 3]), ([2])],
       [([0]), ([0, 1, 2, 3]), ([1, 2]), ([2])],
       [([0, 1]), ([1]), ([1]), ([0, 1, 2, 3])]],
      dtype=object)


In [6]:
print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [7]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)